# MovieLens Reccomendation

>This notebook makes use of a simple library called varname, which easily enables the conversion of variable names into strings, if you have not already installed it uncomment the cell below

In [1]:
#!pip install varname

In [2]:
#Importing necessary libraries
import numpy as np
import pandas as pd
from Code import func, viz
from surprise import Reader, Dataset
from surprise.model_selection import cross_validate, GridSearchCV
from surprise.prediction_algorithms import SVD, KNNWithMeans, KNNBasic, KNNBaseline
import numpy as np

In [3]:
#Reading in data
ratings=pd.read_csv('Data/ratings.csv')
links=pd.read_csv('Data/links.csv')
tags=pd.read_csv('Data/tags.csv')
movies=pd.read_csv('Data/movies.csv')
#See functions.py for details on this function
func.get_names(ratings, links, tags, movies)
df_list = [ratings, links, tags, movies]

We have data spearated into multiple dataframes at the moment. Let's review the data quickly to see what features are where.

In [4]:
#displaying all heads
for df in df_list:
    print(df.__dfname__)
    display(df.head(2))

ratings


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247


links


,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0


tags


,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996


movies


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy


Now that we have an idea of what features we have lets combine them into a dataframe

In [5]:
#join the tables together
df_joined= pd.merge(movies, ratings, on='movieId')
df_joined.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


Since we're working on a baseline model, we're going to drop genres for now, though we will come back to that later. We drop title for performance purposes and can map back to the title when the process is finished. We also drop timestamp here because we're not using it as of yet.

In [27]:
df_new = df_final.drop(['timestamp'], axis=1)
df_new.head()

,movieId,userId,rating,genres_(no genres listed),genres_Action,genres_Adventure,genres_Animation,genres_Children,genres_Comedy,genres_Crime,...,genres_Film-Noir,genres_Horror,genres_IMAX,genres_Musical,genres_Mystery,genres_Romance,genres_Sci-Fi,genres_Thriller,genres_War,genres_Western
0,5,5,20.0,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,5,25,20.0,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,5,35,22.5,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,5,75,12.5,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,5,85,22.5,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0


Reading in dataset into suprise data format. For baseline model we will only include rating, future models will include more 

In [31]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['movieId', 'userId', 'rating']],reader)
dataset = data.build_full_trainset()

In [ ]:
#need to add more to the grid search later
params = {'n_factors': [20, 50, 100],
         'reg_all': [0.02, 0.05, 0.1]}
g_s_svd = GridSearchCV(SVD,param_grid=params,n_jobs=-1)
g_s_svd.fit(data)

print(g_s_svd.best_score)
print(g_s_svd.best_params)

### using KNN


In [ ]:
knn_baseline = KNNBaseline(sim_options={'name':'pearson', 'user_based':True})
cv_knn_baseline = cross_validate(knn_baseline,data)

In [ ]:
for i in cv_knn_baseline.items():
    print(i)

np.mean(cv_knn_baseline['test_rmse'])

In [ ]:
svd = SVD(n_factors= 50, reg_all=0.05)
svd.fit(dataset)
svd.predict(2, 4)

In [ ]:
df_joined.drop(['userId', 'timestamp','rating'], axis=1)

In [ ]:
user_rating = func.movie_rater(df_joined, 3, 'Action')

In [ ]:
user_rating

In [ ]:
new_ratings_df = df_new.append(user_rating,ignore_index=True)
new_data = Dataset.load_from_df(new_ratings_df,reader)

In [ ]:
svd_ = SVD(n_factors= 50, reg_all=0.05)
svd_.fit(new_data.build_full_trainset())

In [ ]:
list_of_movies = []
for m_id in df_new['movieId'].unique():
    list_of_movies.append( (m_id,svd_.predict(1000,m_id)[3]))

In [ ]:
list_of_movies

In [ ]:
ranked_movies = sorted(list_of_movies, key=lambda x:x[1], reverse=True)

In [ ]:
ranked_movies

In [ ]:
func.recommended_movies(ranked_movies,movies,4)